In [1]:
from kiteconnect import KiteConnect
import os
import datetime as dt
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep 
import statsmodels.api as sm
from pyotp import TOTP
from datetime import datetime
from urllib.parse import urlparse,parse_qs
import matplotlib.pyplot as plt
import mplfinance as mpf
import plotly.graph_objects as go
import seaborn as sns

# Automating the kite connect login via selenium

In [2]:
def get_curr_path(folder_name):
    curr_dir = os.getcwd()
    curr_path = os.path.join(curr_dir,folder_name)
    return curr_path

def get_credentials(curr_path,file_name):
    file_dir = os.path.join(curr_path,file_name)
    file = open(file_dir,'r').read().split()
    api_key = file[0]
    api_secret = file[1]
    user_name = file[2]
    pwd = file[3]
    totp_key = file[-1]
    return api_key,api_secret,user_name,pwd,totp_key

def auto_login(api_key,user_name,pwd,totp_key):
    kite = KiteConnect(api_key=api_key)
    service = Service(ChromeDriverManager().install())
    service.start()
    options = Options()
    options.to_capabilities()
    driver = webdriver.Remote(
        command_executor=service.service_url,
        options=options)
    driver.get(kite.login_url())
    driver.implicitly_wait(5)
    username = driver.find_element(By.XPATH, "//input[@type='text']")
    username.send_keys(user_name)
    password = driver.find_element(By.XPATH, "//input[@type='password']")
    password.send_keys(pwd)
    driver.find_element(By.XPATH, "//button[@type='submit']").click()
    sleep(1)
    totp = driver.find_element(By.XPATH,"//input[@type='number']")
    totp_token = TOTP(totp_key)
    token = totp_token.now()
    totp.send_keys(token)
    driver.find_element(By.XPATH,"//button[@type = 'submit']").click()
    sleep(1)
    current_url = driver.current_url
    parsed_url = urlparse(current_url)
    query_params = parse_qs(parsed_url.query)
    request_token = query_params.get('request_token',[None])[0]
    with open('request_token.txt', 'w') as f:
        f.write(request_token)
    request_token = open('request_token.txt','r').read()
    driver.quit()
    return request_token
    

def generate_access_token(request_token,api_key,api_secret):
    request_token = open('request_token.txt','r').read()
    kite = KiteConnect(api_key=api_key)
    data = kite.generate_session(request_token=request_token,api_secret=api_secret)
    data
    with open('access_token.txt','w') as f:
        f.write(data['access_token'])
    access_token = open('access_token.txt','r').read()
    return access_token

In [3]:
folder_name = 'api_keys'
curr_dir = get_curr_path(folder_name)

file_name = 'credentials.txt'
api_key,api_secret,user_name,pwd,totp_key = get_credentials(curr_dir,file_name)

request_token = auto_login(api_key,user_name,pwd,totp_key)

access_token = generate_access_token(request_token,api_key,api_secret)

# Setting up the access token to execute buy,sell orders

In [4]:
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)

In [5]:
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

# Retrieve historical data 

In [6]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1
    
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

def fetchOHLCExtended(ticker, inception_date, interval):
    """Extracts historical data and outputs in the form of a DataFrame.
       inception_date string format - dd-mm-yyyy"""
    instrument = instrumentLookup(instrument_df, ticker)
    from_date = dt.datetime.strptime(inception_date, '%d-%m-%Y')
    data = pd.DataFrame()  # Start with an empty DataFrame
    while True:
        if from_date.date() >= (dt.date.today() - dt.timedelta(100)):
            new_data = pd.DataFrame(kite.historical_data(instrument, from_date, dt.date.today(), interval))
            if not new_data.empty:
                if data.empty:
                    data = new_data
                else:
                    data = pd.concat([data, new_data], ignore_index=True)
            break
        else:
            to_date = from_date + dt.timedelta(100)
            new_data = pd.DataFrame(kite.historical_data(instrument, from_date, to_date, interval))
            if not new_data.empty:
                if data.empty:
                    data = new_data
                else:
                    data = pd.concat([data, new_data], ignore_index=True)
            from_date = to_date
    
    if not data.empty:
        data.set_index("date", inplace=True)
    return data

# Setting up the super trend strategy

In [7]:
def compute_super_trend(df, n, multiplier):
    df['hl2'] = (df['high'] + df['low']) / 2
    df['tr'] = np.maximum.reduce([df['high'] - df['low'],
                                  abs(df['high'] - df['close'].shift(1)),
                                  abs(df['low'] - df['close'].shift(1))])
    alpha = 1 / n

    df.loc[df.index[n-1], 'atr'] = df['tr'][:n].mean()

    for i in range(n, len(df)):
        df.loc[df.index[i], 'atr'] = alpha * df.loc[df.index[i], 'tr'] + (1 - alpha) * df.loc[df.index[i-1], 'atr']

    df['basic_ub'] = df['hl2'] + (multiplier * df['atr'])
    df['basic_lb'] = df['hl2'] - (multiplier * df['atr'])

    df.loc[df.index[n-1], 'ub'] = df.loc[df.index[n-1], 'basic_ub']
    df.loc[df.index[n-1], 'lb'] = df.loc[df.index[n-1], 'basic_lb']

    for i in range(n, len(df)):
        if ((df.loc[df.index[i], 'basic_ub'] < df.loc[df.index[i-1], 'ub']) or 
            (df.loc[df.index[i-1], 'close'] > df.loc[df.index[i-1], 'ub'])):
            df.loc[df.index[i], 'ub'] = df.loc[df.index[i], 'basic_ub']
        else:
            df.loc[df.index[i], 'ub'] = df.loc[df.index[i-1], 'ub']
        
        if ((df.loc[df.index[i], 'basic_lb'] > df.loc[df.index[i-1], 'lb']) or 
            (df.loc[df.index[i-1], 'close'] < df.loc[df.index[i-1], 'lb'])):
            df.loc[df.index[i], 'lb'] = df.loc[df.index[i], 'basic_lb']
        else:
            df.loc[df.index[i], 'lb'] = df.loc[df.index[i-1], 'lb']

    df['trend_direction'] = np.where(df['close'] > df['ub'], 'up', 'down')
    df['super_trend'] = np.where(df['trend_direction'] == 'up', df['lb'], df['ub'])

    for i in range(n, len(df)):
        if df.loc[df.index[i-1], 'trend_direction'] == 'up':
            if df.loc[df.index[i], 'close'] < df.loc[df.index[i], 'lb']:
                df.loc[df.index[i], 'trend_direction'] = 'down'
            else:
                df.loc[df.index[i], 'trend_direction'] = 'up'
        else:
            if df.loc[df.index[i], 'close'] > df.loc[df.index[i], 'ub']:
                df.loc[df.index[i], 'trend_direction'] = 'up'
            else:
                df.loc[df.index[i], 'trend_direction'] = 'down'

        if df.loc[df.index[i], 'trend_direction'] == 'up':
            df.loc[df.index[i], 'super_trend'] = df.loc[df.index[i], 'lb']
        else:
            df.loc[df.index[i], 'super_trend'] = df.loc[df.index[i], 'ub']

    return df[['atr','super_trend']]

In [8]:
ohlc = fetchOHLC('INFY','5minute',5)
ohlc.to_csv('infy.csv')

In [9]:
infy_df = ohlc.copy()

In [10]:
st1 = compute_super_trend(infy_df,7,3)
st2  = compute_super_trend(infy_df,10,3) 
st3 = compute_super_trend(infy_df,11,2)

In [11]:
infy_df[['st1']] = st1
infy_df[['st2']] = st2
infy_df[['st3']] = st3

In [16]:
infy_df.tail(30)

,open,high,low,close,volume,hl2,tr,atr,basic_ub,basic_lb,ub,lb,trend_direction,super_trend,atr_1,st1,atr_2,st2,atr_3,st3
date,,,,,,,,,,,,,,,,,,,,
2024-06-13 13:00:00+05:30,1493.20,1493.50,1492.10,1493.35,43290,1492.800,1.40,1.475944,1495.751889,1489.848111,1495.675164,1492.325358,down,1495.675164,1.421467,1491.336500,1.456216,1492.468593,1.475944,1495.675164
2024-06-13 13:05:00+05:30,1493.35,1493.95,1493.00,1493.15,29601,1493.475,0.95,1.428131,1496.331263,1490.618737,1495.675164,1492.325358,down,1495.675164,1.354114,1491.336500,1.405594,1492.468593,1.428131,1495.675164
2024-06-13 13:10:00+05:30,1493.15,1494.10,1492.50,1493.40,22934,1493.300,1.60,1.443756,1496.187511,1490.412489,1495.675164,1492.325358,down,1495.675164,1.389241,1491.336500,1.425035,1492.468593,1.443756,1495.675164
2024-06-13 13:15:00+05:30,1493.40,1493.50,1492.50,1492.55,21015,1493.000,1.00,1.403414,1495.806829,1490.193171,1495.675164,1492.325358,down,1495.675164,1.333635,1491.336500,1.382531,1492.468593,1.403414,1495.675164
2024-06-13 13:20:00+05:30,1493.00,1493.15,1492.05,1492.20,27045,1492.600,1.10,1.375831,1495.351662,1489.848338,1495.351662,1492.325358,down,1495.351662,1.300259,1491.336500,1.354278,1496.662834,1.375831,1495.351662
2024-06-13 13:25:00+05:30,1492.20,1494.05,1492.05,1494.05,61316,1493.050,2.00,1.432574,1495.915148,1490.184852,1495.351662,1490.184852,down,1495.351662,1.400222,1491.336500,1.418850,1496.662834,1.432574,1495.351662
2024-06-13 13:30:00+05:30,1494.05,1494.15,1492.60,1492.80,29398,1493.375,1.55,1.443249,1496.261498,1490.488502,1495.351662,1490.488502,down,1495.351662,1.421619,1491.336500,1.431965,1496.662834,1.443249,1495.351662
2024-06-13 13:35:00+05:30,1492.75,1493.15,1491.35,1492.15,42844,1492.250,1.80,1.475681,1495.201362,1489.298638,1495.201362,1490.488502,down,1495.201362,1.475673,1491.336500,1.468769,1496.656306,1.475681,1495.201362
2024-06-13 13:40:00+05:30,1492.15,1493.00,1490.60,1491.00,40171,1491.800,2.40,1.559710,1494.919420,1488.680580,1494.919420,1490.488502,down,1494.919420,1.607720,1496.500776,1.561892,1496.485676,1.559710,1494.919420


In [18]:
st_df = infy_df.copy()

In [19]:
st_df.to_csv('st.csv')

# Setting the target profit - using the ATR based approach

In [20]:
def calculate_individual_atr_targets(ohlc,atr_column, multiplier):
    """Calculate target prices based on a single ATR column."""
    atr_value = ohlc[atr_column].iloc[-1]  
    close_price = ohlc['close'].iloc[-1]
    atr_target_price_long = close_price + (atr_value * multiplier)
    atr_target_price_short = close_price - (atr_value * multiplier)
    
    return round(atr_target_price_long, 2), round(atr_target_price_short, 2)

def compute_supertrend_targets(ohlc):
    """Compute target prices based on ATR for each Supertrend line and combine them."""

    atr_columns = ['atr_1', 'atr_2', 'atr_3']
    
    multipliers = [3, 3, 2]
    
    targets_long = []
    targets_short = []
    
    for atr_column, multiplier in zip(atr_columns, multipliers):
        target_long, target_short = calculate_individual_atr_targets(ohlc, atr_column, multiplier)
        targets_long.append(target_long)
        targets_short.append(target_short)
    
    combined_target_long = np.mean(targets_long)
    combined_target_short = np.mean(targets_short)
    
    return round(combined_target_long, 2), round(combined_target_short, 2)

In [22]:
long_target,short_target = compute_supertrend_targets(st_df)

In [34]:
print(f"Long target:{long_target}\nShort target:{short_target}")
print(f"Current closing price:{st_df['close'].iloc[-1]}")

Long target:1499.57
Short target:1486.43
Current closing price:1493.0


# Stop Loss function

In [30]:
def sl_price(ohlc):
    """function to calculate stop loss based on supertrends"""
    st = ohlc.iloc[-1,[-3,-2,-1]]
    if st.min() > ohlc["close"].iloc[-1]:
        st_sorted = st.sort_values(ascending=True)
        sl = (0.6*st_sorted.iloc[0]) + (0.4*st_sorted.iloc[1])
    elif st.max() < ohlc["close"].iloc[-1]:
        st_sorted =  st.sort_values(ascending = False)
        sl = (0.6*st_sorted.iloc[0]) + (0.4*st_sorted.iloc[1])
    else:
        sl = st.mean()
    return round(sl,1)

In [31]:
sl = sl_price(st_df)
print(f"stop loss: {sl}")

stop loss: 1492.3


# Trend Reversal

In [35]:
def st_dir_refresh(ohlc,ticker,st_dir):
    # check for trend reversal
    if (ohlc.loc[ohlc.index[-1],'st1'] > ohlc.loc[ohlc.index[-1],'close']) and (ohlc.loc[ohlc.index[-2],'st1'] < ohlc.loc[ohlc.index[-2],'close']):
        st_dir[ticker][0] = 'red'
    if (ohlc.loc[ohlc.index[-1],'st2'] > ohlc.loc[ohlc.index[-1],'close']) and (ohlc.loc[ohlc.index[-2],'st2'] < ohlc.loc[ohlc.index[-2],'close']):
        st_dir[ticker][1] = 'red'
    if (ohlc.loc[ohlc.index[-1],'st3'] > ohlc.loc[ohlc.index[-1],'close']) and (ohlc.loc[ohlc.index[-2],'st3'] < ohlc.loc[ohlc.index[-2],'close']):
        st_dir[ticker][2] = 'red'
    if (ohlc.loc[ohlc.index[-1],'st1'] < ohlc.loc[ohlc.index[-1],'close']) and (ohlc.loc[ohlc.index[-2],'st1'] > ohlc.loc[ohlc.index[-2],'close']):
        st_dir[ticker][0] = 'green'
    if (ohlc.loc[ohlc.index[-1],'st2'] < ohlc.loc[ohlc.index[-1],'close']) and (ohlc.loc[ohlc.index[-2],'st2'] > ohlc.loc[ohlc.index[-2],'close']):
        st_dir[ticker][1] = 'green'
    if (ohlc.loc[ohlc.index[-1],'st3'] < ohlc.loc[ohlc.index[-1],'close']) and (ohlc.loc[ohlc.index[-2],'st3'] > ohlc.loc[ohlc.index[-2],'close']):
        st_dir[ticker][2] = 'green'
    return st_dir 

In [36]:
st_dir = {}
ticker = 'INFY'
st_dir[ticker] = ['None', 'None', 'None']
# Edge case what if the 2 consecutive days have the same values it would get initialised to None,none,None 
st_dir = st_dir_refresh(st_df, ticker, st_dir)
print(f"Updated st_dir: {st_dir}")

Updated st_dir: {'INFY': ['None', 'None', 'None']}


# Buy Sell function

In [40]:
def placeSLOrder(symbol, buy_sell, quantity, sl_price, target_price):
    """
    Place an intraday stop loss order and a target profit order on NSE.
    
    Parameters:
    - symbol: The trading symbol of the stock.
    - buy_sell: 'buy' for long position or 'sell' for short position.
    - quantity: Number of shares to trade.
    - sl_price: Stop-loss price.
    - target_price: Target profit price.
    """
    if buy_sell == "buy":
        t_type = kite.TRANSACTION_TYPE_BUY
        t_type_sl = kite.TRANSACTION_TYPE_SELL
        t_type_target = kite.TRANSACTION_TYPE_SELL  # Profit-taking is a sell for a long position
    elif buy_sell == "sell":
        t_type = kite.TRANSACTION_TYPE_SELL
        t_type_sl = kite.TRANSACTION_TYPE_BUY
        t_type_target = kite.TRANSACTION_TYPE_BUY  # Profit-taking is a buy for a short position
    
    # Place market order to enter the position
    market_order_id = kite.place_order(
        tradingsymbol=symbol,
        exchange=kite.EXCHANGE_NSE,
        transaction_type=t_type,
        quantity=quantity,
        order_type=kite.ORDER_TYPE_MARKET,
        product=kite.PRODUCT_MIS,
        variety=kite.VARIETY_REGULAR
    )
    
    # Place stop-loss order
    stop_loss_order_id = kite.place_order(
        tradingsymbol=symbol,
        exchange=kite.EXCHANGE_NSE,
        transaction_type=t_type_sl,
        quantity=quantity,
        order_type=kite.ORDER_TYPE_SL,
        price=sl_price,
        trigger_price=sl_price,
        product=kite.PRODUCT_MIS,
        variety=kite.VARIETY_REGULAR
    )
    
    # Place target profit order
    target_order_id = kite.place_order(
        tradingsymbol=symbol,
        exchange=kite.EXCHANGE_NSE,
        transaction_type=t_type_target,
        quantity=quantity,
        order_type=kite.ORDER_TYPE_LIMIT,
        price=target_price,
        product=kite.PRODUCT_MIS,
        variety=kite.VARIETY_REGULAR
    )

    a = 0
    while a < 10:
        try:
            order_list = kite.orders()
            break
        except:
            print("can't get orders..retrying")
            a+=1
    for order in order_list:
        if order["order_id"]==market_order_id:
            if order["status"]=="COMPLETE":
                kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=t_type_sl,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_SL,
                                price=sl_price,
                                trigger_price = sl_price,
                                product=kite.PRODUCT_MIS,
                                variety=kite.VARIETY_REGULAR)
            else:
                kite.cancel_order(order_id=market_order_id,variety=kite.VARIETY_REGULAR)

    
    return market_order_id, stop_loss_order_id, target_order_id

# Example usage:
# symbol = "INFY"
# buy_sell = "buy"
# quantity = 10
# sl_price = 1480.00
# target_price = 1550.00
# placeSLOrder(symbol, buy_sell, quantity, sl_price, target_price)


# Modify order function

In [41]:
def ModifyOrder(order_id,price):    
    # Modify order given order id
    kite.modify_order(order_id=order_id,
                    price=price,
                    trigger_price=price,
                    order_type=kite.ORDER_TYPE_SL,
                    variety=kite.VARIETY_REGULAR)      

In [ ]:
tickers = ['INFY']
capital = 4000

In [ ]:
def main(capital):
    a, b = 0, 0
    while a < 10:
        try:
            pos_df = pd.DataFrame(kite.positions()["day"])
            break
        except:
            print("can't extract position data..retrying")
            a += 1
    while b < 10:
        try:
            ord_df = pd.DataFrame(kite.orders())
            break
        except:
            print("can't extract order data..retrying")
            b += 1
    
    for ticker in tickers:
        print("starting passthrough for.....", ticker)
        try:
            ohlc = fetchOHLC(ticker, "5minute", 4)
            st1 = compute_super_trend(ohlc, 7, 3)
            st2 = compute_super_trend(ohlc, 10, 3)
            st3 = compute_super_trend(ohlc, 11, 2)

            ohlc['st1'] = st1
            ohlc['st2'] = st2
            ohlc['st3'] = st3
            
            # Assuming ATR columns are pre-calculated in OHLC
            st_dir = st_dir_refresh(ohlc, ticker)
            entry_price = ohlc["close"].iloc[-1]
            quantity = int(capital / entry_price)

            # Compute combined ATR-based target prices for the Supertrend lines
            target_long, target_short = compute_supertrend_targets(ohlc, entry_price)

            if len(pos_df.columns) == 0:
                if st_dir[ticker] == ["green", "green", "green"]:
                    market_order_id, stop_loss_order_id, target_order_id = placeSLOrder(
                        ticker, "buy", quantity, sl_price(ohlc), target_long
                    )
                if st_dir[ticker] == ["red", "red", "red"]:
                    market_order_id, stop_loss_order_id, target_order_id = placeSLOrder(
                        ticker, "sell", quantity, sl_price(ohlc), target_short
                    )
            if len(pos_df.columns) != 0 and ticker not in pos_df["tradingsymbol"].tolist():
                if st_dir[ticker] == ["green", "green", "green"]:
                    market_order_id, stop_loss_order_id, target_order_id = placeSLOrder(
                        ticker, "buy", quantity, sl_price(ohlc), target_long
                    )
                if st_dir[ticker] == ["red", "red", "red"]:
                    market_order_id, stop_loss_order_id, target_order_id = placeSLOrder(
                        ticker, "sell", quantity, sl_price(ohlc), target_short
                    )
            if len(pos_df.columns) != 0 and ticker in pos_df["tradingsymbol"].tolist():
                if pos_df[pos_df["tradingsymbol"] == ticker]["quantity"].values[0] == 0:
                    if st_dir[ticker] == ["green", "green", "green"]:
                        market_order_id, stop_loss_order_id, target_order_id = placeSLOrder(
                            ticker, "buy", quantity, sl_price(ohlc), target_long
                        )
                    if st_dir[ticker] == ["red", "red", "red"]:
                        market_order_id, stop_loss_order_id, target_order_id = placeSLOrder(
                            ticker, "sell", quantity, sl_price(ohlc), target_short
                        )
                if pos_df[pos_df["tradingsymbol"] == ticker]["quantity"].values[0] != 0:
                    # Modify existing stop-loss and target orders if the position is already open
                    open_order_ids = ord_df.loc[
                        (ord_df['tradingsymbol'] == ticker) & 
                        (ord_df['status'].isin(["TRIGGER PENDING", "OPEN"])) & 
                        (ord_df['order_type'].isin([kite.ORDER_TYPE_SL, kite.ORDER_TYPE_LIMIT]))
                    ]["order_id"].values
                    
                    for order_id in open_order_ids:
                        order_type = ord_df.loc[ord_df['order_id'] == order_id, 'order_type'].values[0]
                        new_price = sl_price(ohlc) if order_type == kite.ORDER_TYPE_SL else target_long if st_dir[ticker] == ["green", "green", "green"] else target_short
                        ModifyOrder(order_id, new_price, order_type)
        except Exception as e:
            print(f"API error for ticker {ticker}: {e}")

# Squaring off - closing all pending/open orders before the market closes 

In [ ]:
# def placeMarketOrder(symbol,buy_sell,quantity):    
#     # Place an intraday market order on NSE
#     if buy_sell == "buy":
#         t_type=kite.TRANSACTION_TYPE_BUY
#     elif buy_sell == "sell":
#         t_type=kite.TRANSACTION_TYPE_SELL
#     kite.place_order(tradingsymbol=symbol,
#                     exchange=kite.EXCHANGE_NSE,
#                     transaction_type=t_type,
#                     quantity=quantity,
#                     order_type=kite.ORDER_TYPE_MARKET,
#                     product=kite.PRODUCT_MIS,
#                     variety=kite.VARIETY_REGULAR)
    
# def CancelOrder(order_id):    
#     # Modify order given order id
#     kite.cancel_order(order_id=order_id,
#                     variety=kite.VARIETY_REGULAR)  

# #fetching orders and position information   
# a,b = 0,0
# while a < 10:
#     try:
#         pos_df = pd.DataFrame(kite.positions()["day"])
#         break
#     except:
#         print("can't extract position data..retrying")
#         a+=1
# while b < 10:
#     try:
#         ord_df = pd.DataFrame(kite.orders())
#         break
#     except:
#         print("can't extract order data..retrying")
#         b+=1

# #closing all open position      
# for i in range(len(pos_df)):
#     ticker = pos_df["tradingsymbol"].values[i]
#     if pos_df["quantity"].values[i] >0:
#         quantity = pos_df["quantity"].values[i]
#         placeMarketOrder(ticker,"sell", quantity)
#     if pos_df["quantity"].values[i] <0:
#         quantity = abs(pos_df["quantity"].values[i])
#         placeMarketOrder(ticker,"buy", quantity)

# #closing all pending orders
# pending = ord_df[ord_df['status'].isin(["TRIGGER PENDING","OPEN"])]["order_id"].tolist()
# drop = []
# attempt = 0
# while len(pending)>0 and attempt<5:
#     pending = [j for j in pending if j not in drop]
#     for order in pending:
#         try:
#             CancelOrder(order)
#             drop.append(order)
#         except:
#             print("unable to delete order id : ",order)
#             attempt+=1
            